In [ ]:
import numpy as np
import pandas as pd
from scipy import stats, linalg
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# Append base directory
import os,sys
rootname = "conservative-tripartite-testing"
thispath = os.getcwd()
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.metric.dim3d.r2 import pr2_quadratic_triplet_decomp_1D
from mesostat.visualization.mpl_colors import base_colors_rgb

import src.null_models_3D as null3D
import src.null_test as nulltest

%load_ext autoreload
%autoreload 2

In [ ]:
prefixPath = 'figs/discr_vp/'

In [ ]:
discrFuncDict = null3D.discr_method_dict(haveSum=True)

## Noise-dependence

### Effect sizes for example variance

In [ ]:
x = np.random.normal(0, 1, 1000)
y = np.random.normal(0, 1, 1000)
z = np.random.normal(0, 1, 1000)

print(pr2_quadratic_triplet_decomp_1D(x,y,0.3*x+0.7*y))
print(pr2_quadratic_triplet_decomp_1D(x,y,z))

In [ ]:
decompLabels = ['unq_s1', 'unq_s2', 'red', 'syn']
f_metric_discr = lambda x,y,z: dict(zip(decompLabels, pr2_quadratic_triplet_decomp_1D(x,y,z)))

In [ ]:
valThrDict = None
# valThrDict = {'unq_s1': None, 'unq_s2': None, 'red': None, 'syn': 0.28}

In [ ]:
taskDict = {
    'yolo': np.array([0,0,0]),
    'norand': np.array([0,0,0.5]),
    'randx': np.array([0.5,0,0.5]),
    'rand': np.array([0.5,0.5,0.5])
}

for taskName, params in taskDict.items():
    print(taskName)
    rezDict = {}

    # Do continuous tests
    for funcName, func in discrFuncDict.items():
        print('-', funcName)
        
        f_data   = lambda: func(10000, *params)

        rezDF   = nulltest.run_tests(f_data, f_metric_discr, decompLabels, nTest=100)
        rezDFsh = nulltest.run_tests(f_data, f_metric_discr, decompLabels, nTest=100, haveShuffle=True)

        nulltest.plot_test_summary(rezDF, rezDFsh, suptitle=funcName, haveEff=False, valThrDict=valThrDict)
        suffix = '' if valThrDict is None else '_withThr'
        plt.savefig(prefixPath + funcName + '_discr_vp_summary_'+taskName+suffix+'.svg')
        plt.show()

### Effect of variance

In [ ]:
# Do continuous tests
nData = 10000

alphaStratDict = {
    'ImpureX': lambda alpha: [alpha,0,alpha],
    'PureSrc': lambda alpha: [0,0,alpha],
    'Impure' : lambda alpha: [alpha,alpha,alpha],
}

thrMetricDictDict = {
    'H0_orig' : None,
    'H0_adj' : {'unq_s1': 0.1, 'unq_s2': 0.1, 'red': 0.02, 'syn': None}
}

for fName, f_data in discrFuncDict.items():
    for alphaStratName, alphaFunc in alphaStratDict.items():
        # Plot constant thresholds for PureSrc
        avgRand = alphaStratName == 'PureSrc'
        
        f_data_eff = lambda alpha: f_data(nData, *alphaFunc(alpha))
        
        for h0type, thrMetricDict in thrMetricDictDict.items():
            print(fName, alphaStratName, h0type)

            nulltest.run_plot_param_effect(f_data_eff, f_metric_discr, decompLabels, fontsize=12,
                                           nStep=1001, nSkipTest=100, nTest=200, alphaRange=(0, 1),
                                           avgRand=avgRand, thrMetricDict=thrMetricDict, plotAlphaSq=False)

            suffix = 'n_' + str(nData) + '_' + alphaStratName + '_' + h0type

            plt.savefig(prefixPath + fName + '_discr_vp_scatter_vareff_'+suffix+'.svg')
            plt.show()

In [ ]:
nData=10000
for fName, f_data in contFuncDict.items():
    print(fName)
    
    f_data_eff = lambda alpha: f_data(n=nData, aX=alpha, aY=alpha, aZ=alpha)
    nulltest.run_plot_param_effect_test(f_data_eff, f_metric_cont, decompLabels,
                                        nStep=10, nTest=400, alphaRange=(0, 1), valThrDict=valThrDict)
    
    suffix = '' if valThrDict is None else '_withThr'
    plt.savefig(prefixPath + fName + '_r2_vareff_n'+str(nData)+suffix+'.png', dpi=200)
    plt.show()

### Effect of number of samples

In [ ]:
alpha=0.25

alphaStratDict = {
    'PureSrc': [0,0,alpha],
    'ImpureX': [alpha,0,alpha],
    'Impure' : [alpha,alpha,alpha],
}

nDataLstRed, thrRandLstRed, thrAdjLstRed = np.loadtxt(prefixPath + 'discr_vp_msum_1Dscan_red_summary.csv', dtype=float)
nDataLstUnq, thrRandLstUnq, thrAdjLstUnq = np.loadtxt(prefixPath + 'discr_vp_mred_1Dscan_unq_summary.csv', dtype=float)
# nDataLstSyn, thrRandLstSyn, thrAdjLstSyn = np.loadtxt(prefixPath + 'discr_mmi_mred_1Dscan_syn_summary.csv', dtype=float)
thrDictRed = dict(zip(nDataLstRed, thrAdjLstRed))
thrDictUnq = dict(zip(nDataLstUnq, thrAdjLstUnq))
# thrDictSyn = dict(zip(nDataLstSyn, thrAdjLstSyn))

thrMetricDictDict = {
    'H0_orig' : None,
#     'H0_adj'  : {'unq_s1': 0.195, 'unq_s2': 0.195, 'red': None, 'syn': None}
    'H0_adj' : {'unq_s1': thrDictUnq, 'unq_s2': thrDictUnq, 'red': thrDictRed, 'syn': None}
}


for fName, f_data in discrFuncDict.items():
    for alphaStratName, alphaFunc in alphaStratDict.items():
        f_data_eff = lambda n: f_data(n, *alphaFunc)

        for h0type, thrMetricDict in thrMetricDictDict.items():
            print(fName, alphaStratName, h0type)

            nulltest.run_plot_data_effect(f_data_eff, f_metric_discr, decompLabels,
                                          nStep=101, nSkipTest=10, nTest=200, pVal=0.01,
                                          thrMetricDict=thrMetricDict, fontsize=12)

            suffix = 'sig_' + str(alpha) + '_' + alphaStratName + '_' + h0type

            plt.savefig(prefixPath + fName + '_discr_vp_scatter_nEff_'+suffix+'.svg')
            plt.show()

In [ ]:
alpha=0.5
for fName, f_data in contFuncDict.items():
    print(fName)

    f_data_eff = lambda n: f_data(n=n, aX=alpha, aY=alpha, aZ=alpha)
    nulltest.run_plot_data_effect_test(f_data_eff, f_metric_cont, decompLabels,
                                       nStep=10, nTest=400, valThrDict=valThrDict)
    
    suffix = '' if valThrDict is None else '_withThr'
    plt.savefig(prefixPath + fName + '_r2_nEff_sig'+str(sig)+suffix+'.png', dpi=200)
    plt.show()

# Computing conservative thresholds

In [ ]:
tableauColors = base_colors_rgb(key='tableau')
tableauMap = dict(zip(decompLabels, tableauColors[:4]))

## Phase 1: Testing all noise parameter combinations

We try to find patterns for lines in 3D where largest FP emerge

### MRed -> Unq

Find noise fraction combination that causes highest FP unique atoms
* NOTE: Scan only over $p_x = p_y$. In case of mismatched noise fractions FP unique arises naturally, but this is a problem in experimental design rather than testing so it is not included in this testing procedure.
* NOTE 2: Actual assumption in this case is that noise is equal in all tested channels. Hence it is ok to simply test $p_x = p_y = p_z$ only

**Intermediate Conclusion**: Largest noise for $p_x=p_y=0.5$, $p_z = 0$

In [ ]:
f_data_2D = lambda nData, a, b: null3D.discr_red_noisy(nData, a, a, b)

paramArr3D, dataArr3D = nulltest.run_scan_bare(f_data_2D, f_metric_discr, 2, decompLabels, varLimits=(0, 1),
                                               nData=5000, nStep=30, nTest=20)

In [ ]:
nulltest.print_scan_max(paramArr3D, dataArr3D, 'unq_s1', decompLabels, nMax=10)

In [ ]:
nulltest.plot_scan_2D(dataArr3D, 'unq_s1', decompLabels, 30, (0,1), fontsize=16, haveColorbar=True)

### MRed -> Syn

**Intermediate Conclusion**: Results stochastic, likely non-significant

In [ ]:
f_data_3D = lambda nData, a, b, c: null3D.discr_red_noisy(nData, a, b, c)

paramArr3D, dataArr3D = nulltest.run_scan_bare(f_data_3D, f_metric_discr, 3, decompLabels, varLimits=(0, 1),
                                               nData=5000, nStep=10, nTest=20)

In [ ]:
nulltest.print_scan_max(paramArr3D, dataArr3D, 'syn', decompLabels, nMax=10)

In [ ]:
nulltest.plot_scan_3D_2D_bytrg(paramArr3D, dataArr3D, 'syn', decompLabels, 5, 10, (0, 1),
                               fontsize=16, haveColorbar=True)

### MUnq -> Red

**Intermediate Conclusion**: Radially symmetric, lower noise better, possibly insignificant

In [ ]:
f_data_2D = lambda nData, a, b: null3D.discr_unq_noisy(nData, a, 0, b)

paramArr2D, dataArr2D = nulltest.run_scan_bare(f_data_2D, f_metric_discr, 2, decompLabels, varLimits=(0, 1),
                                               nData=5000, nStep=30, nTest=20)

In [ ]:
nulltest.plot_scan_2D(dataArr2D, 'red', decompLabels, 30, (0,1), fontsize=16, haveColorbar=True)

### MUnq -> Syn

**Intermediate Conclusion**: Radially symmetric, higher noise better, possibly insignificant

In [ ]:
nulltest.plot_scan_2D(dataArr2D, 'syn', decompLabels, 30, (0,1), fontsize=16, haveColorbar=True)

### MSyn -> Red

**Intermediate Conclusion**:  Erratic, insignificant

In [ ]:
f_data_3D = lambda nData, a, b, c: null3D.discr_syn_noisy(nData, a, b, c)

paramArr3D, dataArr3D = nulltest.run_scan_bare(f_data_3D, f_metric_discr, 3, decompLabels, varLimits=(0, 1),
                                               nData=5000, nStep=10, nTest=20)

In [ ]:
nulltest.print_scan_max(paramArr3D, dataArr3D, 'red', decompLabels, nMax=10)

In [ ]:
nulltest.plot_scan_3D_2D_bytrg(paramArr3D, dataArr3D, 'red', decompLabels, 0, 10, (0, 1),
                               fontsize=16, haveColorbar=True)

### MSyn -> Unq

**Intermediate Conclusion**: Erratic, insignificant

In [ ]:
nulltest.print_scan_max(paramArr3D, dataArr3D, 'unq_s1', decompLabels, nMax=10)

In [ ]:
nulltest.plot_scan_3D_2D_bytrg(paramArr3D, dataArr3D, 'unq_s1', decompLabels, 2, 10, (0, 1),
                               fontsize=16, haveColorbar=True)

# Phase 1.1: Correction for composite effects

### MUnq -> Red: Use $Z = X + Y$ as a more general null model

**Intermediate Conclusion**: Radial in all directions, likely maximal at zero noise

In [ ]:
f_data_3D = lambda nData, a, b, c: null3D.discr_sum_noisy(nData, a, b, c)

paramArr3D, dataArr3D = nulltest.run_scan_bare(f_data_3D, f_metric_discr, 3, decompLabels, varLimits=(0, 1),
                                               nData=5000, nStep=10, nTest=20)

In [ ]:
nulltest.print_scan_max(paramArr3D, dataArr3D, 'red', decompLabels, nMax=10)

In [ ]:
nulltest.plot_scan_3D_2D_bytrg(paramArr3D, dataArr3D, 'red', decompLabels, 0, 10, (0, 1),
                               fontsize=16, haveColorbar=True)

In [ ]:
# Verify using radial search
f_data_2D = lambda nData, a, b: null3D.discr_sum_noisy(nData, a, a, b)

paramArr2D, dataArr2D = nulltest.run_scan_bare(f_data_2D, f_metric_discr, 2, decompLabels, varLimits=(0, 1),
                                               nData=5000, nStep=20, nTest=100)

In [ ]:
nulltest.plot_scan_2D(dataArr2D, 'red', decompLabels, 20, (0,1), fontsize=16, haveColorbar=True)

### MUnq -> Syn

**Intermediate Conclusion**: Erratic, likely insignificant

In [ ]:
nulltest.print_scan_max(paramArr3D, dataArr3D, 'syn', decompLabels, nMax=10)

# Phase 2: Actual testing

In [ ]:
loopLst = [
    ['red', 'unq', 'red',    'unq_s1', lambda nData, alpha: null3D.discr_red_noisy(nData, alpha, alpha, alpha)],
    ['red', 'syn', 'red',    'syn',    lambda nData, alpha: null3D.discr_red_noisy(nData, alpha, alpha, alpha)],
    ['unq', 'red', 'unq_s1', 'red',    lambda nData, alpha: null3D.discr_unq_noisy(nData, alpha, alpha, alpha)],
    ['unq', 'syn', 'unq_s1', 'syn',    lambda nData, alpha: null3D.discr_unq_noisy(nData, alpha, alpha, alpha)],
    ['sum', 'red', 'unq_s1', 'red',    lambda nData, alpha: null3D.discr_sum_noisy(nData, alpha, alpha, alpha)],
    ['sum', 'syn', 'unq_s1', 'syn',    lambda nData, alpha: null3D.discr_sum_noisy(nData, alpha, alpha, alpha)],
    ['syn', 'red', 'syn',    'red',    lambda nData, alpha: null3D.discr_syn_noisy(nData, alpha, alpha, alpha)],
    ['syn', 'unq', 'syn',    'unq_s1', lambda nData, alpha: null3D.discr_syn_noisy(nData, alpha, alpha, alpha)]
]

In [ ]:
nSample = 1000
nDataLst = (10**np.linspace(2, 4, 10)).astype(int)
# nDataLst = [10000]
for labelA, labelB, atomA, atomB, f_data_1D in loopLst:
    prefix = prefixPath + 'discr_vp_m'+labelA+'_1Dscan_'+labelB
    thrAdjLst = []
    thrRandLst = []
    
    for nData in nDataLst:
        paramArr1D, dataArr1D = nulltest.run_scan_bare(f_data_1D, f_metric_discr, 1, decompLabels,
                                                       varLimits=(0, 1), nData=nData, nStep=100, nTest=20)

        thrAdj = nulltest.resample_get_thr(f_data_1D, f_metric_discr, atomB, decompLabels, paramArr1D, dataArr1D,
                                           nData=nData, nTestResample=nSample, pVal=0.01, haveShuffle=False)

        thrShuffle = nulltest.resample_get_thr(f_data_1D, f_metric_discr, atomB, decompLabels, paramArr1D, dataArr1D,
                                               nData=nData, nTestResample=nSample, pVal=0.01, haveShuffle=True)
        
        print(nData, thrAdj, thrShuffle)

        savename = prefix+'_n_'+str(nData)+'.svg'
        nulltest.plot_scan_1D(paramArr1D, dataArr1D, [atomA, atomB], atomB, decompLabels,
                              maxThr=thrAdj, colorDict=tableauMap, savename=savename,
                              fontsize=16, xlabel='Noise Fraction', ylabel='Relative SSE')
        plt.close()

        thrAdjLst += [thrAdj]
        thrRandLst += [thrShuffle]

    plt.figure()
#     plt.plot(nDataLst, alphaMaxLst)
    plt.plot(nDataLst, thrAdjLst, label='adjusted', color='purple')
    plt.plot(nDataLst, thrRandLst, label='shuffle')
    plt.legend()
    plt.ylim([0, None])
    plt.savefig(prefix + '_summary.svg')
    plt.show()
    
    # Save results to file
    rezArr = np.array([nDataLst, thrRandLst, thrAdjLst])
    np.savetxt(prefix + '_summary.csv', rezArr)

#### 3. Determining Unique-Redundancy Relationship

In [ ]:
discrDataMethodDict = {
    'Cont' : null3D.cont_method_dict(),
    'Discr' : null3D.discr_method_dict()
}

atomCombList = {
    ['red',    'unq_s1'],
    ['red',    'syn'],
    ['unq_s1', 'red'],
    ['unq_s1', 'syn'],
    ['syn',    'red'],
    ['syn',    'unq_s1']
}

In [ ]:
for discrKey, dataMethodsDict in discrDataMethodDict.items():
    for fDataLabel, f_data_3D in dataMethodsDict.items():
        for atomA, atomB in atomCombList:
            nulltest.run_plot_scatter_explore(f_data_3D, f_metric_cont,
                                              atomA, atomB, 3,
                                              varLimits=(0, 1), nData=1000, nTestDim=20)

# Determining testing thresholds for real data

In [ ]:
import h5py

In [ ]:
# Only test combinations that matter
loopLst = [
    ['red', 'unq', 'red',    'unq_s1', lambda nData, alpha: null3D.cont_red_noisy(nData, alpha, alpha, alpha)],
    ['unq', 'red', 'unq_s1', 'red',    lambda nData, alpha: null3D.cont_unq_noisy(nData, alpha, alpha, alpha)],
]

# TEX + AUD
nDataLst = [1315, 1209, 3967, 1910, 1724, 4784, 1307, 1324, 5191, 1132, 1014, 3111] + \
           [1070, 510, 2498, 1274, 735, 3407, 1918, 953, 4472, 1008, 630, 2320] + \
           [564, 591, 605, 643, 812, 1040, 1131, 1166, 1263, 1317, 1406, 1412, 1448, 1525, 1668, 1974, 2438, 2767, 2891, 3228, 3278, 7106, 8209]


In [ ]:
for labelA, labelB, atomA, atomB, f_data_1D in loopLst:
    print(labelA, labelB)

    for nData in nDataLst:
        key = labelA + '_' + labelB + '_' + str(nData)
        with h5py.File('pr2_rand_dist.h5', 'a') as h5f:
            if key in h5f.keys():
                print(key, 'already done')
                continue
        
        print(key)
        
        randValues = nulltest.run_1D_scan_bare(f_data_1D, f_metric_cont, atomB,
                                               varLimits=(0, 1), nData=nData,
                                               nStep=100, nTest=100, nTestResample=10000)[1]
        
        with h5py.File('pr2_rand_dist.h5', 'a') as h5f:
            h5f[key] = randValues